### Instalação dos pacotes necessários

Pandas: Biblioteca para manipulação e análise de dados
PyArrow: Biblioteca para leitura e escrita de arquivos parquet
Spotipy: Biblioteca para acesso a API do Spotify

Use o comando abaixo para instalar os pacotes necessários:
```bash
$ pip install pandas pyarrow spotipy
```

Ou instale as dependências diretamente do arquivo `requirements.txt`:
```bash
$ pip install -r requirements.txt
```

### Configuração da API do Spotify com um .env

Para acessar a API do Spotify é necessário criar um aplicativo no [Spotify for Developers](https://developer.spotify.com/dashboard/applications) e obter as credenciais de acesso.

Crie um arquivo `.env` na raiz do projeto e adicione as seguintes variáveis de ambiente:
```bash
SPOTIPY_CLIENT_ID=seu_client_id
SPOTIPY_CLIENT_SECRET=sua_client_secret
```

In [292]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import glob

In [293]:
env_vars = {}

# Le o arquivo .env
with open('.env') as f:
    env_vars = dict(
        tuple(line.replace('"', '').replace("'", '').strip().split('=', 1)) for line in f
    )

# Cria o objeto de autenticação
client_credentials_manager = SpotifyClientCredentials(
    client_id=env_vars['SPOTIPY_CLIENT_ID'],
    client_secret=env_vars['SPOTIPY_CLIENT_SECRET']
)

# Configurando as credenciais do cliente
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [294]:
# Obtém as playlists de um determinado termo de busca e salva em um arquivo
# parquet
def get_playlists(
    query = '',
    directory = '',
    n = 1,
    offset = 50,
    market = 'BR',
    verbose = False,
):
    print(f'{directory} - obtendo playlists...')
    all_playlists = []
    play_ids = []
    for i in range(n):
        try:
            playlists = sp.search(
                query,
                limit=offset,
                offset=i*offset,
                type='playlist',
                market = market,
            )
            # Printando o nome das playlists
            for item in playlists['playlists']['items']:
                play_tracks = 0
                if verbose:
                    print(f"Playlist encontrada: {item['name']}")
                all_playlists.append(item)
                play_ids.append(item['id'])
        except Exception as e:
            if 'http status: 404' in str(e):
                print("Playlist não encontrada.")
                break
            elif 'http status: 400' in str(e):
                print(
                    "Erro 400: Provavelmente o offset é maior que o número "
                    "de playlists disponíveis."
                )
                break
            else:
                print("Erro ao obter playlists: ", e)
                continue
    # Salvando o objeto de playlists em um arquivo parquet
    df = pd.DataFrame(all_playlists)
    # Convertendo o DataFrame para um Arrow Table
    table = pa.Table.from_pandas(df)
    # Salvando o arquivo parquet
    pq.write_table(table, directory + '/playlists.parquet')
    print(f'{directory} - playlists obtidas: {len(play_ids)}')
    return play_ids

# Função para obter todas as músicas de uma playlist
def get_playlist_tracks(
    play_id,
    directory = '',
    n = 1,
    offset = 50,
    market = 'BR',
):
    print(f"{play_id} - obtendo músicas...")
    all_tracks = []
    for i in range(n):
        try:
            results = sp.playlist_tracks(
                play_id,
                limit = offset,
                offset=i*offset,
                additional_types=('track'),
            )
            if not results['items']:
                print("Playlist vazia.")
                break
            all_tracks.extend(results['items'])
        except Exception as e:
            if 'http status: 404' in str(e):
                print("Playlist não encontrada.")
                break
            elif 'http status: 400' in str(e):
                print(
                    "Erro 400: Provavelmente o offset é maior que o número "
                    "de músicas disponíveis."
                )
                break
            else:
                print("Erro ao obter músicas da playlist: ", e)
                continue
    # Salvando o objeto de tracks em um arquivo parquet
    df = pd.DataFrame(all_tracks)
    # Convertendo o DataFrame para um Arrow Table
    table = pa.Table.from_pandas(df)
    # Salvando o arquivo parquet
    pq.write_table(table, directory + f'/tracks_{play_id}.parquet')
    print(f"{play_id} - músicas obtidas: {len(all_tracks)}")



In [295]:
# Unificar todos os arquivos parquet de tracks do diretório
def unify_track_files(directory, filename = None):

    # Lista todos os arquivos parquet do diretório
    files = glob.glob(directory + '/tracks_*.parquet')

    for file in files:
        filename = file.split('/')[-1].replace('.parquet', '')

        table = pq.read_table(file)
        df = table.to_pandas()

        new_df = pd.DataFrame()

        for row in df.iterrows():
            if not row[1]['track']:
                continue
            # Converte cada linha em uma entrada na tabela com colunas
            new_row = {
                'id': row[1]['track']['id'] if row[1]['track']['id'] else None,
                'nome': row[1]['track']['name'] if row[1]['track']['name'] else None,
                'popularidade': row[1]['track']['popularity'] if row[1]['track']['popularity'] else None,
                'track': row[1]['track']['track'] if row[1]['track']['track'] else None,
                'num_track': row[1]['track']['track_number'] if row[1]['track']['track_number'] else None,
                'tipo': row[1]['track']['type'] if row[1]['track']['type'] else None,
                'album': row[1]['track']['album']['name']
                    if row[1]['track']['album'] and row[1]['track']['album']['name'] else None,
                'artistas': [{'nome': artist['name'], 'id': artist['id']}
                    if artist and artist['name'] and artist['id'] else None
                    for artist in row[1]['track']['artists']],
                'num_disc': row[1]['track']['disc_number'] if row[1]['track']['disc_number'] else None,
                'duracao': row[1]['track']['duration_ms'] if row[1]['track']['duration_ms'] else None,
                'explicita': row[1]['track']['explicit'] if row[1]['track']['explicit'] else None,
                'ids_externos': row[1]['track']['external_ids'] if row[1]['track']['external_ids'] else None,
                'urls_externas': row[1]['track']['external_urls'] if row[1]['track']['external_urls'] else None,
                'href': row[1]['track']['href'] if row[1]['track']['href'] else None,
                'local': row[1]['track']['is_local'] if row[1]['track']['is_local'] else None,
                'preview_url': row[1]['track']['preview_url'] if row[1]['track']['preview_url'] else None,
                'uri': row[1]['track']['uri'] if row[1]['track']['uri'] else None,
            }
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

        # Exportando o DataFrame para um novo arquivo parquet
        table = pa.Table.from_pandas(new_df)
        pq.write_table(table, directory + '/pre_proc_' + filename + '.parquet')

In [296]:
def unify_playlist_files(directory):
    table = pq.read_table(directory + '/playlists.parquet')
    df = table.to_pandas()

    new_df = pd.DataFrame()

    for row in df.iterrows():
        if not row[1]['id']:
            continue
        # Converte cada linha em uma entrada na tabela com colunas
        # Nome da Playlist, Descrição, Link Externo, Proprietário, Total de Faixas
        
        new_row = {
            'id': row[1]['id']
                if row[1]['id'] else None,
            'nome': row[1]['name']
                if row[1]['name'] else None,
            'descricao': row[1]['description']
                if row[1]['description'] else None,
            'link': row[1]['external_urls']['spotify']
                if row[1]['external_urls'] and row[1]['external_urls']['spotify'] else None,
            'proprietario': row[1]['owner']['display_name']
                if row[1]['owner'] and row[1]['owner']['display_name'] else None,
            'n_faixas': row[1]['tracks']['total']
                if row[1]['tracks'] and row[1]['tracks']['total'] else None,
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

    # Exportando o DataFrame para um novo arquivo parquet
    table = pa.Table.from_pandas(new_df)
    pq.write_table(table, directory + '/pre_proc.parquet')

### Execução do script

Usando a lista de queries fornecida, o script irá buscar playlists no Spotify e salvar os dados em um arquivo parquet
relacionado a cada query.

Além disso, para cada playlist encontrada, o script irá buscar as músicas e salvar os dados em um arquivo parquet
relacionado a cada playlist.

In [297]:
import time

estilo_1 = 'rock'
estilo_2 = 'pop'

# Lista de queries para buscar playlists
# Essa lista vai definir quais playlists serão buscadas
# e vai dividir as rodadas em pastas diferentes
query_list = [
   # estilo_1 + ' independente brasil',
   # estilo_1 + ' indie brasil',
   # estilo_1 + ' alternativo brasil',
   # estilo_1 + ' paradas brasil',
   estilo_1 + ' top hits brasil',
   estilo_1 + ' lançamentos brasil',
   estilo_2 + ' independente brasil',
   estilo_2 + ' indie brasil',
   estilo_2 + ' alternativo brasil',
   estilo_2 + ' paradas brasil',
   estilo_2 + ' top hits brasil',
   estilo_2 + ' lançamentos brasil',
]

for query in query_list:
    # Criar a pasta para a query
    directory = 'queries/' + query.replace(' ', '_')
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Obter as playlists
    playlists = get_playlists(
        query, directory = directory,
        n = 2, offset = 50,
        market = 'BR'
    )
    # Obter as músicas das playlists
    for play_id in playlists:
        get_playlist_tracks(
            play_id,
            directory = directory,
            n = 5, offset = 50,
            market = 'BR'
        )
        time.sleep(10)
        
    # Unificar os arquivos parquet de tracks e playlists
    unify_playlist_files(directory)
    unify_track_files(directory)

queries/rock_top_hits_brasil - obtendo playlists...
queries/rock_top_hits_brasil - playlists obtidas: 100
1HO90ioxlve4CkBrJYM6Tn - obtendo músicas...
1HO90ioxlve4CkBrJYM6Tn - músicas obtidas: 218
37i9dQZEVXbMXbN3EUUhlg - obtendo músicas...
Playlist vazia.
37i9dQZEVXbMXbN3EUUhlg - músicas obtidas: 50
6QhzuDMMhMICdc6JBzcn5b - obtendo músicas...
6QhzuDMMhMICdc6JBzcn5b - músicas obtidas: 250
0n3ydoZmAIehk3Vu3NWLRa - obtendo músicas...
Playlist vazia.
0n3ydoZmAIehk3Vu3NWLRa - músicas obtidas: 162
0WTDmgqpxkkPpAE067lRNs - obtendo músicas...
Playlist vazia.
0WTDmgqpxkkPpAE067lRNs - músicas obtidas: 44
42Gj0MPuyT5R3EjZIx3S1p - obtendo músicas...
Playlist vazia.
42Gj0MPuyT5R3EjZIx3S1p - músicas obtidas: 195
7pLtdA2bZEF3QHZqJ4oi2L - obtendo músicas...
7pLtdA2bZEF3QHZqJ4oi2L - músicas obtidas: 211
2NzZ9Z1vF45Gmkywg0BJ91 - obtendo músicas...
Playlist vazia.
2NzZ9Z1vF45Gmkywg0BJ91 - músicas obtidas: 154
71jVsitRDQ8OUgZRpIPVHM - obtendo músicas...
Playlist vazia.
71jVsitRDQ8OUgZRpIPVHM - músicas obt

/tmp/ipykernel_7312/604964690.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)


queries/rock_lançamentos_brasil - obtendo playlists...
queries/rock_lançamentos_brasil - playlists obtidas: 100
5x2Cps2k22PkpZy1WzJM9Y - obtendo músicas...
Playlist vazia.
5x2Cps2k22PkpZy1WzJM9Y - músicas obtidas: 104
7wqENcBYxYZO5iHtfKv8WA - obtendo músicas...
7wqENcBYxYZO5iHtfKv8WA - músicas obtidas: 250
1PvTq1vJUFyO41qMZfvePB - obtendo músicas...
Playlist vazia.
1PvTq1vJUFyO41qMZfvePB - músicas obtidas: 97
3zgTHeyU9HLpUKQ0nBc1EK - obtendo músicas...
Playlist vazia.
3zgTHeyU9HLpUKQ0nBc1EK - músicas obtidas: 100
5j68ojJTu85SyJ135zKL9H - obtendo músicas...
Playlist vazia.
5j68ojJTu85SyJ135zKL9H - músicas obtidas: 101
5BNaNcoYn20SY4VjUOGXL2 - obtendo músicas...
Playlist vazia.
5BNaNcoYn20SY4VjUOGXL2 - músicas obtidas: 99
5rDEAggtyJ9QlnCxEVGCCX - obtendo músicas...
Playlist vazia.
5rDEAggtyJ9QlnCxEVGCCX - músicas obtidas: 79
3O0kKyRVt6TxYBTU4EYikO - obtendo músicas...
Playlist vazia.
3O0kKyRVt6TxYBTU4EYikO - músicas obtidas: 143
5URoUbk6wuXA6min6Qkms0 - obtendo músicas...
Playlist vazia.